 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Theano-Implementation" data-toc-modified-id="Theano-Implementation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theano Implementation</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Keras-implementation" data-toc-modified-id="Keras-implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Keras implementation</a></span></li></ul></div>

# EMOS Network

In [42]:
# Imports
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
import utils; reload(utils)
from utils import *

In [23]:
# Basic setup
DATA_DIR = '/Users/stephanrasp/repositories/ppnn/data/'  # Mac
fn = 'data_interpolated.nc'
window_size = 50
fclt = 48
utc = 0

## Theano Implementation

In [31]:
model = EMOS_Network()

In [19]:
date_idx = 1507

In [24]:
tobs_full, tfc_full, dates = load_nc_data(DATA_DIR + fn, utc=utc)

In [35]:
tobs_full.shape

(3653, 537)

In [25]:
tfc_mean_train, tfc_std_train, tobs_train, \
    tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, 
                            window_size, fclt)

In [26]:
lr = np.asarray(0.1, dtype='float32')
early_stopping_delta = 0.000001

In [33]:
model.fit(tfc_mean_train, tfc_std_train, tobs_train, 1000, 
          (tfc_mean_test, tfc_std_test, tobs_test), lr=lr, 
          early_stopping_delta=early_stopping_delta)

Step 0; mean_crps = 1.169
Step 50; mean_crps = 1.169
Stop training at step 62


(array(1.168672504823591), array(1.0251625654654968))

In [47]:
train_crps_list, valid_crps_list = loop_over_days(
    tobs_full, 
    tfc_full, 
    100, 120, 
    window_size=25,
    fclt=48,     
    epochs_max=2000, 
    early_stopping_delta=1e-6, 
    lr=0.1)

100
120
Time: 4.98 s


## Keras implementation

In [49]:
import keras

Using TensorFlow backend.


In [ ]:
# Using Tensorflow backend is ok because that way I can use the GPU's at work
